In [8]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [9]:
pip install pillow


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import random

# Constants
lattice_size = 20  # Lattice size (NxN)
n_steps = 10000
mus_H = np.linspace(-0.2, 0, 7)
Ts = np.linspace(0.001, 0.019, 7)

# Convert T values to Kelvin (assuming T in eV/k_B, where k_B = 8.6173e-5 eV/K)
k_B = 8.6173e-5
Ts_kelvin = Ts / k_B

# Interaction parameters (eV)
interaction_sets = {
    "Ideal Mixture": [-0.1, -0.1, 0.0, 0.0, 0.0],
    "Repulsive Interactions": [-0.1, -0.1, 0.05, 0.05, 0.05],
    "Attractive Interactions": [-0.1, -0.1, -0.05, -0.05, -0.05],
    "Immiscible": [-0.1, -0.1, -0.05, -0.05, 0.05],
    "Like Dissolves Unlike": [-0.1, -0.1, 0.05, 0.05, -0.05]
}

# Function to initialize lattice
def initialize_lattice(size):
    lattice = np.zeros((size, size))  # 0 represents empty sites
    return lattice

# Function to compute neighbor indices
def compute_neighbor_indices(size):
    neighbor_indices = {}
    for x in range(size):
        for y in range(size):
            neighbors = [
                ((x - 1) % size, y),
                ((x + 1) % size, y),
                (x, (y - 1) % size),
                (x, (y + 1) % size)
            ]
            neighbor_indices[(x, y)] = neighbors
    return neighbor_indices

# Function to calculate interaction energy
def calculate_interaction_energy(lattice, site, particle, neighbor_indices, epsilon_HH, epsilon_NN, epsilon_NH):
    x, y = site
    interaction_energy = 0
    for neighbor in neighbor_indices[(x, y)]:
        neighbor_particle = lattice[neighbor]
        if neighbor_particle != 0:
            if particle == 1:  # Particle H
                if neighbor_particle == 1:
                    interaction_energy += epsilon_HH
                else:
                    interaction_energy += epsilon_NH
            else:  # Particle N
                if neighbor_particle == 2:
                    interaction_energy += epsilon_NN
                else:
                    interaction_energy += epsilon_NH
    return interaction_energy

# Function to attempt a Monte Carlo move
def attempt_move(lattice, N_H, N_N, H_empty, neighbor_indices, params):
    size = lattice.shape[0]
    H_sites = size * size
    beta = 1 / params['T']
    epsilon_H, epsilon_N, epsilon_HH, epsilon_NN, epsilon_NH, mu_H, mu_N = (
        params['epsilon_H'], params['epsilon_N'], params['epsilon_HH'],
        params['epsilon_NN'], params['epsilon_NH'], params['mu_H'], params['mu_N']
    )

    if random.random() < 0.5:  # Attempt to add a particle
        if H_empty == 0:
            return N_H, N_N, H_empty
        site = tuple(random.choice(np.argwhere(lattice == 0)))
        particle = random.choice([1, 2])
        if particle == 1:
            mu = mu_H
            epsilon = epsilon_H
            N_s = N_H
        else:
            mu = mu_N
            epsilon = epsilon_N
            N_s = N_N
        delta_E = epsilon + calculate_interaction_energy(lattice, site, particle, neighbor_indices, epsilon_HH, epsilon_NN, epsilon_NH)
        acc_prob = min(1, (H_empty / (N_s + 1)) * np.exp(-beta * (delta_E - mu)))
        if random.random() < acc_prob:
            lattice[site] = particle
            if particle == 1:
                N_H += 1
            else:
                N_N += 1
            H_empty -= 1
    else:  # Removing a particle
        if H_sites - H_empty == 0:
            return N_H, N_N, H_empty
        site = tuple(random.choice(np.argwhere(lattice != 0)))
        particle = lattice[site]
        if particle == 1:
            mu = mu_H
            epsilon = epsilon_H
            N_s = N_H
        else:
            mu = mu_N
            epsilon = epsilon_N
            N_s = N_N
        delta_E = -epsilon - calculate_interaction_energy(lattice, site, particle, neighbor_indices, epsilon_HH, epsilon_NN, epsilon_NH)
        acc_prob = min(1, (N_s / (H_empty + 1)) * np.exp(-beta * (delta_E + mu)))
        if random.random() < acc_prob:
            lattice[site] = 0
            if particle == 1:
                N_H -= 1
            else:
                N_N -= 1
            H_empty += 1
    return N_H, N_N, H_empty

# Function to plot lattice configuration
def plot_lattice(ax, lattice, title):
    size = lattice.shape[0]
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])

    # Plot particles as circles
    for x in range(size):
        for y in range(size):
            if lattice[x, y] == 1:
                ax.add_patch(plt.Circle((y, x), 0.3, color='blue'))
            elif lattice[x, y] == 2:
                ax.add_patch(plt.Circle((y, x), 0.3, color='red'))

    # Add grid lines
    for x in range(size + 1):
        ax.plot([x - 0.5, x - 0.5], [-0.5, size - 0.5], color='black', lw=0.5)
    for y in range(size + 1):
        ax.plot([-0.5, size - 0.5], [y - 0.5, y - 0.5], color='black', lw=0.5)
    ax.set_xlim(-0.5, size - 0.5)
    ax.set_ylim(-0.5, size - 0.5)
    ax.set_aspect('equal')

# Update function for animation
def update_lattice(frame, lattice, ax, neighbor_indices, params):
    ax.clear()
    plot_lattice(ax, lattice, title=f"Frame {frame}")

    # Perform multiple Monte Carlo moves to update lattice for this frame
    for _ in range(100):  # Update every 100 moves for smoother animation
        N_H, N_N, H_empty = attempt_move(
            lattice, params['N_H'], params['N_N'], params['H_empty'], neighbor_indices, params
        )
        params['N_H'], params['N_N'], params['H_empty'] = N_H, N_N, H_empty

# Function to create animation
def create_animation(interaction_name, params):
    fig, ax = plt.subplots(figsize=(6, 6))
    lattice = initialize_lattice(lattice_size)
    neighbor_indices = compute_neighbor_indices(lattice_size)
    params.update({'N_H': 0, 'N_N': 0, 'H_empty': lattice_size ** 2})

    # Setup animation using FuncAnimation
    anim = FuncAnimation(
        fig, update_lattice, frames=n_steps // 100, fargs=(lattice, ax, neighbor_indices, params),
        interval=200, repeat=False
    )
    
    # Save the animation as a gif
    anim.save(f"{interaction_name}_adsorption.gif", writer="pillow", fps=10)
    plt.close()  # Close the figure to avoid displaying in real-time

# Generate and save animations for each interaction type
for interaction_name, interaction_params in interaction_sets.items():
    mu_H = mus_H[3]  # Choose a middle value for mu_H
    T = Ts[3]  # Choose a middle value for T
    params = {
        'epsilon_H': interaction_params[0],
        'epsilon_N': interaction_params[1],
        'epsilon_HH': interaction_params[2],
        'epsilon_NN': interaction_params[3],
        'epsilon_NH': interaction_params[4],
        'mu_H': mu_H,
        'mu_N': -0.1,
        'T': T
    }
    create_animation(interaction_name, params)
